In [72]:
import exiftool
import glob
import sqlite3
import json

In [68]:
DATADIR = '/home/aubrey/Desktop'
DATE = '20201001'

In [69]:
conn = sqlite3.connect('test.db')
c = conn.cursor()
c.execute("CREATE TABLE exif (exif json)")

In [82]:
for video in glob.glob(f'{DATADIR}/{DATE}/{DATE}_??????.mp4'):
    print(video)
    with exiftool.ExifTool() as et:
        meta = et.get_metadata(video)
    #jmeta = json.dumps(meta)
    #sql = f'INSERT INTO exif VALUES ({jmeta});'
    c.execute("INSERT INTO exif VALUES (?)", [json.dumps(meta)])
    conn.commit()  

/home/aubrey/Desktop/20201001/20201001_102824.mp4
/home/aubrey/Desktop/20201001/20201001_095445.mp4
/home/aubrey/Desktop/20201001/20201001_092426.mp4


In [91]:
sql='''
    SELECT 
        JSON_EXTRACT(exif, "$.QuickTime:CreateDate") AS start_time_utc,
        JSON_EXTRACT(exif, "$.QuickTime:Duration") AS duration    
    FROM exif;
    '''
c.execute(sql)

rows = c.fetchall()

for row in rows:
    print(row)

('2020:10:01 01:05:31', 1986.661)
('2020:10:01 00:31:46', 2019.305)
('2020:09:30 23:58:07', 2019.373)


In [55]:
pattern = f'"{DATADIR}/{DATE}/"'
print(pattern)
glob.glob(pattern)

"/home/aubrey/Desktop/survey1/20201001/"


[]

In [ ]:
with exiftool.ExifTool() as et:
    createDate = et.get_tag('QuickTime:CreateDate', filename)
    logging.info(f'Exif Quicktime:CreateDate: {createDate} UTC')
    duration = et.get_tag('QuickTime:Duration', filename)
    logging.info(f'Exif QuickTime:Duration: {duration} seconds')


In [3]:
import sqlite3

db_file = 'data/raw/db.sqlite'
tables = {
    'Players': {
        'id': 'INTEGER PRIMARY KEY',
        'fname': 'TEXT',
        'lname': 'TEXT',
        'dob': 'DATETIME',
        'age': 'INTEGER',
        'height': 'INTEGER', # inches
        'weight': 'INTEGER', # pounds
        'rank': 'INTEGER',
        'rhlh': 'INTEGER', # 0 - right, 1 - left
        'bh': 'INTEGER', # 0 - onehand, 1 - twohand
        'city': 'TEXT', # birth city
        'county': 'TEXT' #birth country
        }
}


conn = sqlite3.connect(db_file)
c = conn.cursor()

for table in tables.keys():
    fieldset = []
    for col, definition in tables[table].items():
        fieldset.append("'{0}' {1}".format(col, definition))

    if len(fieldset) > 0:
        query = "CREATE TABLE IF NOT EXISTS {0} ({1})".format(table, ", ".join(fieldset))

        c.execute(query)

c.close()
conn.close()